###Importing libraries

In [1]:
import numpy as np
import pandas as pd
import re
import nltk

###Importing dataset (from google drive)

In [22]:
dataset = pd.read_csv('Restaurant reviews.csv')
dataset.drop('Rating', inplace=True, axis=1)
dataset.drop('Unnamed: 1', inplace=True, axis=1)
dataset.head()

,Restaurant,Review
0,Beyond Flavours,"The ambience was good, food was quite good . h..."
1,Beyond Flavours,Ambience is too good for a pleasant evening. S...
2,Beyond Flavours,A must try.. great food great ambience. Thnx f...
3,Beyond Flavours,Soumen das and Arun was a great guy. Only beca...
4,Beyond Flavours,Food is good.we ordered Kodi drumsticks and ba...


###Data cleaning

In [23]:


nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

all_stopwords = stopwords.words('english')
all_stopwords.remove('not')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\apraj\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [24]:
corpus=[]

for i in range(0, 10000):
  review = re.sub('[^a-zA-Z]', ' ', str(dataset['Review'][i]))
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
  review = ' '.join(review)
  corpus.append(review)

###Data transformation

In [25]:
# Loading BoW dictionary
from sklearn.feature_extraction.text import CountVectorizer
import pickle
cvFile='c1_BoW_Sentiment_Model.pkl'
# cv = CountVectorizer(decode_error="replace", vocabulary=pickle.load(open('./drive/MyDrive/Colab Notebooks/2 Sentiment Analysis (Basic)/3.1 BoW_Sentiment Model.pkl', "rb")))
cv = pickle.load(open(cvFile, "rb"))


In [26]:
X_fresh = cv.transform(corpus).toarray()
X_fresh.shape

(10000, 1420)

###Predictions (via sentiment classifier)

In [27]:
import joblib
classifier = joblib.load('c2_Classifier_Sentiment_Model')
classifier_rf = joblib.load('c2_Classifier_Sentiment_Model_rf')
classifier_SVC = joblib.load('c2_Classifier_Sentiment_Model_SVC')

In [28]:
y_pred = classifier.predict(X_fresh)
y_pred_rf = classifier_rf.predict(X_fresh)
y_pred_SVC = classifier_SVC.predict(X_fresh)

In [29]:
dataset['predicted_label_nb'] = y_pred.tolist()
dataset['predicted_label_rf'] = y_pred_rf.tolist()
dataset['predicted_label_svc'] = y_pred_SVC.tolist()


dataset.head()

,Restaurant,Review,predicted_label_nb,predicted_label_rf,predicted_label_svc
0,Beyond Flavours,"The ambience was good, food was quite good . h...",0,1,1
1,Beyond Flavours,Ambience is too good for a pleasant evening. S...,1,1,1
2,Beyond Flavours,A must try.. great food great ambience. Thnx f...,1,1,1
3,Beyond Flavours,Soumen das and Arun was a great guy. Only beca...,1,1,1
4,Beyond Flavours,Food is good.we ordered Kodi drumsticks and ba...,1,1,1


In [30]:
dataset.to_csv("c4_Predicted_Sentiments_Fresh_Dump.csv", encoding='UTF-8', index=False)